# STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성
- 데이터셋은 깃허브에서 다운받거나, [Huggingface datasets](https://huggingface.co/datasets)에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

In [1]:
import datasets
from datasets import load_dataset, Dataset


# huggingface_nsmc_dataset = load_dataset('nsmc')
# print(huggingface_nsmc_dataset)

train_data = load_dataset('nsmc', split='train[:7%]')
val_data = load_dataset('nsmc' ,split='train[-3%:]')
test_data = load_dataset('nsmc' ,split='test[:9%]')

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)
Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)
Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


In [2]:
train_data

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 10500
})

In [3]:
val_data

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 4500
})

In [4]:
test_data

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 4500
})

### 데이터 전처리

In [5]:
# import pandas as pd
# train_df = pd.DataFrame(train_data, columns=['id', 'document', 'label'])

In [6]:
# train_df['document'].nunique(), train_df['label'].nunique()

In [7]:
# train_df.drop_duplicates(subset=['document'], inplace=True)

In [8]:
# print('총 샘플의 수 :',len(train_df))

In [9]:
# train_df['label'].value_counts().plot(kind='bar')

In [10]:
# print(train_df.groupby('label').size().reset_index(name='count'))

In [11]:
# print(train_df.isnull().values.any())

In [12]:
# # 한글과 공백을 제외하고 모두 제거
# train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# train_df[:5]

In [13]:
# import numpy as np

# # 공백만 있는 값들을 Null로 변경
# train_df['document'] = train_df['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
# train_df['document'].replace('', np.nan, inplace=True)
# print(train_df.isnull().sum())


In [14]:
# train_df.loc[train_df.document.isnull()][:5]


In [15]:
# test_df = pd.DataFrame(test_data, columns=['id', 'document', 'label'])

In [16]:
# test_df.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
# test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
# test_df['document'] = test_df['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
# test_df['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
# test_df = test_df.dropna(how='any') # Null 값 제거
# print('전처리 후 테스트용 샘플의 개수 :',len(test_df))

In [17]:
# val_df = pd.DataFrame(val_data, columns=['id', 'document', 'label'])

In [18]:
# val_df.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
# val_df['document'] = val_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
# val_df['document'] = val_df['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
# val_df['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
# val_df = val_df.dropna(how='any') # Null 값 제거
# print('전처리 후 테스트용 샘플의 개수 :',len(val_df))

In [19]:
# train_dataset = Dataset.from_pandas(train_df)

# # 변환 함수 적용
# # transformed_dataset = train_dataset.map(transform, batched=True)

In [20]:
# train_dataset

In [21]:
# train = huggingface_nsmc_dataset['train']
# cols = train.column_names
# cols

In [22]:
# # for i in range(5):
#     for col in cols:
#         print(col, ':', train[col][i])
#     print('\n')

nsmc에는 train, 과 test로만 구성된것을 확인
그리고 데이터 속 colum들도 확인한 결과 id, document, label 로 구성된것을 확인했다.

# STEP 2. klue/bert-base model 및 tokenizer 불러오기

In [23]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


# model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-small", num_labels=2)
# tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [24]:
def transform(data):
    return tokenizer(
        data['document'],
        truncation=True,
        padding='max_length',
        return_token_type_ids=False,)

In [25]:
train_data = train_data.map(transform, batched=True)
test_data = test_data.map(transform, batched=True)
val_data = val_data.map(transform, batched=True)

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-a442af94df917b74.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-c0c2cd4ffd724df1.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-ddef61b6a2450974.arrow


In [26]:
# hf_dataset = huggingface_nsmc_dataset.map(transform, batched=True)

# # train & validation & test split
# # import random

# # 무작위로 90000개의 데이터만 선택
# hf_train_dataset = hf_dataset['train'][:]
# # hf_train_dataset = hf_train_dataset.select(indices)
# hf_test_dataset = hf_dataset['test'][:25000]
# hf_val_dataset = hf_dataset['test'][25000:]

In [27]:
# hf_test_dataset

In [28]:
# hf_train_dataset

# STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기

In [29]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv("HOME") + '/aiffel/aiffel_quest/mini_quest_240326'

training_arguments = TrainingArguments(
output_dir,                        # output이 저장될 경로
evaluation_strategy='epoch',       # evaluation하는 빈도
learning_rate=2e-5,                # learning_rate
per_device_train_batch_size=14,     # 각 device 당 batch size(8)
per_device_eval_batch_size=14,      # evaluation 시에 batch size(8)
num_train_epochs=5,                # train 시킬 총 epochs
weight_decay=0.01)                 # weight decay



In [30]:
# from datasets import load_metric
# f1_metric = load_metric("f1")
# accuracy_metric = load_metric("accuracy")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
    
#     # F1 점수 계산
#     f1_result = f1_metric.compute(predictions=predictions, references=labels)
    
#     # Accuracy 계산
#     accuracy_result = accuracy_metric.compute(predictions=predictions, references=labels)
    
#     return {"f1": f1_result, "accuracy": accuracy_result}


from datasets import load_metric
metric = load_metric('accuracy')


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)






In [31]:
import torch

# 코드 실행 중간에 GPU 메모리를 해제해야 하는 경우
torch.cuda.empty_cache()


In [32]:
%%time
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 10500
  Num Epochs = 5
  Instantaneous batch size per device = 14
  Total train batch size (w. parallel, distributed & accumulation) = 14
  Gradient Accumulation steps = 1
  Total optimization steps = 3750


Epoch,Training Loss,Validation Loss,Accuracy
1,0.373700,0.287949,0.877111
2,0.219700,0.306724,0.877333
3,0.125200,0.552791,0.880889
4,0.060100,0.698543,0.877778
5,0.037700,0.711291,0.879778


Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 14
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1500/config.json
Model w

CPU times: user 1h 20min 5s, sys: 9min 36s, total: 1h 29min 42s
Wall time: 1h 29min 51s


TrainOutput(global_step=3750, training_loss=0.16356908645629883, metrics={'train_runtime': 5386.8061, 'train_samples_per_second': 9.746, 'train_steps_per_second': 0.696, 'total_flos': 1.38133304064e+16, 'train_loss': 0.16356908645629883, 'epoch': 5.0})

In [33]:
trainer.evaluate(test_data)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 14


{'eval_loss': 0.7022930383682251,
 'eval_accuracy': 0.8782222222222222,
 'eval_runtime': 146.1483,
 'eval_samples_per_second': 30.791,
 'eval_steps_per_second': 2.203,
 'epoch': 5.0}

# STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
- 데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

In [34]:
import gc
del training_arguments, trainer, model
gc.collect()

38

In [35]:
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-small", num_labels=2)
# tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/klue/bert-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transform

In [36]:
output_dir = os.getenv("HOME") + '/aiffel/aiffel_quest/mini_quest_240326'

training_arguments = TrainingArguments(
output_dir,                        # output이 저장될 경로
evaluation_strategy='epoch',       # evaluation하는 빈도
learning_rate=5e-5,                # learning_rate
per_device_train_batch_size=14,     # 각 device 당 batch size(8)
per_device_eval_batch_size=14,      # evaluation 시에 batch size(8)
num_train_epochs=3,                # train 시킬 총 epochs
weight_decay=0.01)                 # weight decay



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [37]:
%%time
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 10500
  Num Epochs = 3
  Instantaneous batch size per device = 14
  Total train batch size (w. parallel, distributed & accumulation) = 14
  Gradient Accumulation steps = 1
  Total optimization steps = 2250


Epoch,Training Loss,Validation Loss,Accuracy
1,0.386900,0.327417,0.865111
2,0.215600,0.345009,0.876222
3,0.093100,0.558512,0.878889


Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 14
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1500/config.json
Model w

CPU times: user 47min 58s, sys: 5min 46s, total: 53min 45s
Wall time: 54min 35s


TrainOutput(global_step=2250, training_loss=0.22607358127170138, metrics={'train_runtime': 3275.6901, 'train_samples_per_second': 9.616, 'train_steps_per_second': 0.687, 'total_flos': 8287998243840000.0, 'train_loss': 0.22607358127170138, 'epoch': 3.0})

In [38]:
trainer.evaluate(test_data)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 14


{'eval_loss': 0.5776839256286621,
 'eval_accuracy': 0.8722222222222222,
 'eval_runtime': 145.7294,
 'eval_samples_per_second': 30.879,
 'eval_steps_per_second': 2.21,
 'epoch': 3.0}

# STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교
- 아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.
    - [Data Collator](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator)
    - [Trainer.TrainingArguments 의 group_by_length](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)
- STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.

In [44]:
del training_arguments, trainer, model, tokenizer
gc.collect()

18

In [45]:

model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-small", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small", use_fast=True)


loading configuration file https://huggingface.co/klue/roberta-small/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/6aa0817a5b48e36ec821c3d9cb8267f2dda41a32b317311688f06c742b6b2a1d.7f009d6b27d54554dce063a5fc8273742e9a198f6f627aada4bc5ea2dbbc0313
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.11.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}

loading weigh

In [48]:
output_dir = os.getenv("HOME") + '/aiffel/aiffel_quest/mini_quest_240326'

training_arguments = TrainingArguments(
output_dir,                        # output이 저장될 경로
evaluation_strategy='epoch',       # evaluation하는 빈도
learning_rate=5e-5,                # learning_rate
per_device_train_batch_size=10,     # 각 device 당 batch size(8)
per_device_eval_batch_size=10,      # evaluation 시에 batch size(8)
num_train_epochs=5,                # train 시킬 총 epochs
group_by_length=True,
weight_decay=0.01)                 # weight decay


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [49]:
%%time
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 10500
  Num Epochs = 5
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 5250


Epoch,Training Loss,Validation Loss,Accuracy
1,0.421300,0.363625,0.849333
2,0.329500,0.491835,0.850889
3,0.208200,0.608962,0.854889
4,0.130700,0.659481,0.858000
5,0.083100,0.782902,0.853111


Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 10
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1500/config.json
Mode

CPU times: user 45min 23s, sys: 16.5 s, total: 45min 40s
Wall time: 46min 32s


TrainOutput(global_step=5250, training_loss=0.24319228744506835, metrics={'train_runtime': 2787.9867, 'train_samples_per_second': 18.831, 'train_steps_per_second': 1.883, 'total_flos': 6954538429440000.0, 'train_loss': 0.24319228744506835, 'epoch': 5.0})

In [50]:
from transformers import DataCollatorWithPadding, DataCollatorWithPadding


# 토크나이저와 데이터 콜레이터 정의
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base", use_fast=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https://huggingface.co/klue/bert-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/1a36e69d48a0

In [70]:
def transform(data):
    encoded_data = tokenizer(
        data['document'],
        truncation=True,
        padding='max_length',
        max_length=30,
        return_tensors="pt",
        return_token_type_ids=False,)
    
    # 텐서를 리스트로 변환하여 반환
    return {key: value.squeeze().tolist() for key, value in encoded_data.items()}


train_data = train_data.map(transform)#, batched=True)
test_data = test_data.map(transform)#, batched=True)
val_data = val_data.map(transform)#, batched=True)

  0%|          | 0/10500 [00:00<?, ?ex/s]

  0%|          | 0/4500 [00:00<?, ?ex/s]

  0%|          | 0/4500 [00:00<?, ?ex/s]

In [74]:
output_dir = os.getenv("HOME") + '/aiffel/aiffel_quest/mini_quest_240326'

training_arguments = TrainingArguments(
output_dir,                        # output이 저장될 경로
evaluation_strategy='epoch',       # evaluation하는 빈도
learning_rate=2e-5,                # learning_rate
per_device_train_batch_size=10,     # 각 device 당 batch size(8)
per_device_eval_batch_size=10,      # evaluation 시에 batch size(8)
num_train_epochs=20,                # train 시킬 총 epochs
group_by_length=True,
weight_decay=0.01)                 # weight decay

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
%%time
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 10500
  Num Epochs = 20
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 21000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.035500,1.116465,0.822222
2,0.045200,1.182217,0.831556
3,0.043600,1.103943,0.840889
4,0.039200,1.192465,0.830667
5,0.043600,1.175388,0.830889
6,0.025300,1.223011,0.839778
7,0.019200,1.170813,0.839111
8,0.022200,1.289116,0.839333
9,0.012200,1.337091,0.830889
10,0.016700,1.337301,0.839778


Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 10
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-1500/config.json
Mode

***** Running Evaluation *****
  Num examples = 4500
  Batch size = 10
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-11000
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-11000/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-11000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-11500
Configuration saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-11500/config.json
Model weights saved in /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-11500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 10
Saving model checkpoint to /aiffel/aiffel/aiffel_quest/mini_quest_240326/checkpoint-12000
Configuration saved 

In [ ]:
trainer.evaluate(test_data)

Bucketing 이란?<br>
NLP에서의 Bucketing은 데이터를 효율적으로 처리하기 위한 전략 중 하나입니다. 특히, Sequence-to-Sequence 모델을 학습할 때 사용되며, 입력 문장의 길이에 따라 데이터를 그룹화하여 배치를 구성하는 방법입니다.

일반적으로, 서로 다른 길이의 문장을 동일한 배치에 함께 처리할 때 모든 문장의 길이를 동일하게 맞추는 것이 일반적입니다. 그러나 이는 GPU의 메모리를 비효율적으로 사용할 수 있습니다. 예를 들어, 모든 배치에 포함된 문장의 최대 길이보다 더 긴 문장이 있을 경우, 모든 문장을 최대 길이에 맞추어 패딩(padding)을 추가해야 합니다. 이렇게 되면 패딩된 토큰들은 실제 의미를 가지지 않지만 연산에도 참여하게 되어 메모리와 계산 시간을 낭비하게 됩니다.

Bucketing은 이러한 문제를 해결하기 위한 방법 중 하나입니다. Bucketing은 데이터를 여러 그룹으로 나누고, 각 그룹 내에서는 비슷한 길이의 문장들을 함께 배치로 만듭니다. 이렇게 하면 패딩의 양을 최소화하고 효율적으로 GPU 메모리를 사용할 수 있습니다.

예를 들어, Bucketing을 사용하여 문장의 길이에 따라 다음과 같이 데이터를 그룹화할 수 있습니다:

길이가 1-10 토큰인 문장들의 그룹
길이가 11-20 토큰인 문장들의 그룹
길이가 21-30 토큰인 문장들의 그룹
...
그런 다음 각 그룹에서는 비슷한 길이의 문장들을 배치로 만들어 처리합니다. 이렇게 함으로써 효율적인 학습이 가능해집니다.

Bucketing은 특히 Transformer와 같은 Self-Attention 기반의 모델을 학습할 때 유용합니다. Transformer 모델의 Self-Attention 계산은 시퀀스 길이에 따라 계산량이 크게 달라지기 때문에, Bucketing을 통해 비슷한 길이의 문장들을 함께 처리하여 계산 효율을 높일 수 있습니다.

Dynamic padding이란?<br>
Dynamic padding은 패딩을 적용할 때, 각 배치에 포함된 샘플들의 길이에 맞게 패딩을 조절하는 기법입니다. 정적 패딩(static padding)과는 달리, 각 배치마다 서로 다른 길이의 샘플을 가지고 있을 때, 최대한 적은 양의 패딩을 사용하여 효율적으로 메모리를 사용합니다.

일반적으로 정적 패딩은 모든 샘플을 특정 길이로 맞추기 위해 최대 길이에 맞게 패딩을 적용하는 반면, 동적 패딩은 각 배치에 포함된 샘플 중 가장 긴 샘플의 길이에 맞게 패딩을 적용합니다. 이렇게 하면 모든 샘플이 동일한 길이로 패딩되는 것이 아니라, 각 샘플의 실제 길이에 맞게 패딩이 적용되어 메모리를 더 효율적으로 사용할 수 있습니다.

동적 패딩은 주로 RNN(Recurrent Neural Network)이나 Transformer와 같은 시퀀스 모델에서 사용됩니다. 이러한 모델들은 입력 시퀀스의 길이가 다를 수 있으며, 동적 패딩은 이러한 모델에서 효율적인 배치 처리를 위해 중요한 역할을 합니다. Hugging Face의 Transformers 라이브러리에서도 동적 패딩을 쉽게 적용할 수 있습니다.

# 회고
